Below is the step wise procedure of training and testing the transformer models.

## Setting the Base Directory


In [ ]:
BASE_DIR="/home/x/programs/my_project"

## Import training and testing CSV files 
from ERENO dataset ("https://www.kaggle.com/datasets/sequincozes/ereno-iec61850-ids") into

{BASE_DIR}/data/raw

## Preprocess Raw Data

To train weak classifiers, baselines and transformers.



In [ ]:
%cd {BASE_DIR}/scr/preprocessing/preprocessing_raw
%run main_preprocessing.py --config preprocessing_transformers_config.json --base_path {BASE_DIR}
%run main_preprocessing.py --config preprocessing_weak_classifiers_config.json --base_path {BASE_DIR}

## Perform Multimixing
for Transformers and Weak Classifiers

Set tasks (data diversity), Length (length of multimixed dataset), and Z_Constant (define distribution of multimixed dataset)

Note. In the scripts, we have predefined In Distribution data

In [ ]:
tasks = 700
Length = 150000
Z_Constant = 0.65

In [ ]:
%cd {BASE_DIR}/scr/preprocessing/mult_mixing
%run multi_mixing.py --config training_transformers_multi_mixing_config.json --tasks {tasks} --Length {Length} --Z_Constant {Z_Constant} --base_path {BASE_DIR}
%run multi_mixing.py --config training_weak_classifiers_multi_mixing_config.json --tasks {tasks} --Length {Length} --Z_Constant {Z_Constant} --base_path {BASE_DIR}


## Train Weak Classifiers
for Transformers Training


In [ ]:
tasks = 700
Z_Constant = 0.65

num_models = 30
batch_size = 256
lr = 0.0005
num_epochs = 3

%cd {BASE_DIR}/scr/train_weak_classifiers
%run main.py --config config.json --batch_size {batch_size} --learning_rate {lr} --num_epochs {num_epochs} --tasks {tasks} --Z_Constant {Z_Constant} --num_models {num_models} --base_path {BASE_DIR}

## Generate Labels/Probability Distributions for Transformer Training
from trained weak classifiers

Note. Initially we generate only distribution. Later we use argmax function to convert them into hard labels

In [ ]:
tasks = 700
Z_Constant = 0.65

batch_size_dist = 5000 # We randomly select weak classifiers in batches

%cd {BASE_DIR}/scr/preprocessing/run_weak_classifiers
%run run_weak_classifiers.py --config run_weak_classifiers_config.json --base_path {BASE_DIR} --tasks {tasks} --batch_size {batch_size_dist} --Z_Constant {Z_Constant}

## Train Transformer Model

In [ ]:
tasks = 700
Length = 200000
Z_Constant = 0.65

tf_num_models = 1
tf_batch_size = 200
tf_epochs = 4

%cd {BASE_DIR}/scr/train_transformers

## Distributional Transformer (DTF)

Mix Trained DTF (MDTF)

In [ ]:
%run main.py --config mdtf_config.json --base_path {BASE_DIR} --tasks {tasks} --Z_Constant {Z_Constant} --batch_size {tf_batch_size} --num_models {tf_num_models} --num_epochs {tf_epochs}


Weak Classifier Trained DTF (WCTF)

In [ ]:
%run main.py --config wcdtf_config.json --base_path {BASE_DIR} --tasks {tasks} --Z_Constant {Z_Constant} --batch_size {tf_batch_size} --num_models {tf_num_models} --num_epochs {tf_epochs}


## Simple Transformer (TF)

In [16]:
tasks = 700
Z_Constant = 0.65

tf_num_models = 1
tf_batch_size = 200
tf_epochs = 4

Mix Trained TF (MTF)

In [ ]:
%run main.py --config mtf_config.json --base_path {BASE_DIR} --tasks {tasks} --Z_Constant {Z_Constant} --batch_size {tf_batch_size} --num_models {tf_num_models} --num_epochs {tf_epochs}


Weak Classifier Trained TF (WCTF)

In [ ]:
%run main.py --config wctf_config.json --base_path {BASE_DIR} --tasks {tasks} --Z_Constant {Z_Constant} --batch_size {tf_batch_size} --num_models {tf_num_models} --num_epochs {tf_epochs}


## Train Weak Classifiers for testing

note. The least task diversity we have tested on is 100.

Hence we test all transformer models (for all tesk diversity) with weak classifiers trained on task diversity 100.

In [ ]:
testing_wc_tasks = 100
testing_wc_Length = 100000
testing_wc_Z_Constant = 0.55

%cd {BASE_DIR}/scr/preprocessing/mult_mixing
%run multi_mixing.py --config training_weak_classifiers_multi_mixing_config.json --tasks {testing_wc_tasks} --Length {testing_wc_Length} --Z_Constant {testing_wc_Z_Constant} --base_path {BASE_DIR}


In [ ]:
testing_wc_num_models = 10
testing_wc_batch_size = 250
testing_wc_lr = 0.0005
testing_wc_num_epochs = 3

%cd {BASE_DIR}/scr/train_weak_classifiers
%run main.py --config config.json --batch_size {testing_wc_batch_size} --learning_rate {testing_wc_lr} --num_epochs {testing_wc_num_epochs} --tasks {testing_wc_tasks} --Z_Constant {testing_wc_Z_Constant} --num_models {testing_wc_num_models} --base_path {BASE_DIR}

In [ ]:

%cd {BASE_DIR}/scr/preprocessing/run_weak_classifiers
%run run_weak_classifiers_test.py --config run_weak_classifiers_test_config.json --base_path {BASE_DIR} --tasks {testing_wc_tasks} --Z_Constant {testing_wc_Z_Constant}

## Test Transformer Models

In [ ]:
testing_wc_tasks = 100
testing_wc_Z_Constant = 0.55

testing_tf_tasks = 700
testing_tf_Z_Constant = 0.65


%cd {eval('BASE_DIR')}/test/transformers

## Distributional Transformer

Mix Trained DTF (MDTF)

In [ ]:
%run main.py --config mdtf_config.json --base_path {BASE_DIR} --wc_tasks {testing_wc_tasks} --wc_Z_Constant {testing_wc_Z_Constant} --tf_tasks {testing_tf_tasks} --tf_Z_Constant {testing_tf_Z_Constant}


Weak Classifier Trained DTF (WCDTF)

In [ ]:
%run main.py --config wcdtf_config.json --base_path {BASE_DIR} --wc_tasks {testing_wc_tasks} --wc_Z_Constant {testing_wc_Z_Constant} --tf_tasks {testing_tf_tasks} --tf_Z_Constant {testing_tf_Z_Constant}


## Simple Transformer (TF)

Mix Trained TF (MTF)

In [ ]:
%run main.py --config mtf_config.json --base_path {BASE_DIR} --wc_tasks {testing_wc_tasks} --wc_Z_Constant {testing_wc_Z_Constant} --tf_tasks {testing_tf_tasks} --tf_Z_Constant {testing_tf_Z_Constant}

Weak Classifier Trained TF (WCTF)

In [ ]:
%run main.py --config wctf_config.json --base_path {BASE_DIR} --wc_tasks {testing_wc_tasks} --wc_Z_Constant {testing_wc_Z_Constant} --tf_tasks {testing_tf_tasks} --tf_Z_Constant {testing_tf_Z_Constant}
